In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
!{sys.executable} -m pip install tensorflow-gpu tensorflow_datasets tensorflow_text

  Using cached tensorflow_gpu-2.8.0-cp37-cp37m-manylinux2010_x86_64.whl (497.5 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [5]:
import sys

# Use this if running this notebook from within its place in the truera repository.
sys.path.insert(0, "..")

# Install transformers / huggingface.
# !{sys.executable} -m pip install torch
# !{sys.executable} -m pip install transformers

# Or otherwise install trulens.
# !{sys.executable} -m pip install git+https://github.com/truera/trulens.git

from IPython.display import display
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text 

In [2]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


2022-05-05 17:28:21.236461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 17:28:21.237318: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 17:28:21.242072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 17:28:21.242880: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 17:28:21.243670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [3]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

2022-05-05 17:28:22.338214: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-05 17:28:22.490213: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 17:28:22.491263: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-05 17:28:22.492036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [8]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(2)))
train_examples_batch

2022-05-05 17:17:35.341747: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [108]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [139]:
print(preprocessor.outbound_nodes[0].layer)
print(encoder.inbound_nodes[0].layer)

In [213]:
ids = encoder.get_input_at(0)['input_word_ids']

In [227]:
n = ids.node
kl = n.outbound_layer

In [231]:
kl.get_output_at(0)

{'input_mask': <KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'keras_layer_4')>,
 'input_word_ids': <KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'keras_layer_4')>,
 'input_type_ids': <KerasTensor: shape=(None, 128) dtype=int32 (created by layer 'keras_layer_4')>}

In [145]:
encoder.

[<tf.Variable 'Variable:0' shape=() dtype=bool, numpy=True>]

In [111]:
kmodel = tf.keras.Model(text_input, sequence_output)

In [146]:
imported = tf.saved_model.load("/home/piotrm/tfhub_modules/d760773f85f64fc84ae0b47310f7cfe3bcec4868")

In [187]:
f = imported.signatures['serving_default']
f

<ConcreteFunction signature_wrapper(*, input_type_ids, input_mask, input_word_ids) at 0x7F49DC64E588>

In [205]:
res = f.captured_inputs[0]
res.shape

TensorShape([])

In [180]:
f.graph.get_operation_by_name("model/bert_encoder/word_embeddings/Reshape_1")

KeyError: "The name 'model/bert_encoder/word_embeddings/Reshape_1' refers to an Operation not in the graph."

In [172]:
f.graph.as_graph_def()#get_tensor_by_name("model/bert_encoder/word_embeddings/Reshape_1/output:0")

node {
  name: "input_mask"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "input_mask"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "input_type_ids"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "input_type_ids"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "input_word_ids"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "input_word_ids"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        di

In [162]:
encoded = preprocessor(tf.constant(['this is a sentence', 'and so is this']))
out = f(**encoded)

In [163]:
with tf.GradientTape() as gt:
    # l1 = out['bert_encoder_1']

<tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
array([[[ 0.15590554, -0.01118364, -0.07785353, ...,  0.06163237,
          0.03383987,  0.06297513],
        [-0.47564048,  0.5763171 ,  0.13226584, ..., -0.05813633,
          0.41404092,  0.04797438],
        [-0.9628004 , -0.28790185, -0.43244758, ...,  0.12608324,
          0.17628382,  0.36824045],
        ...,
        [ 0.0069404 , -0.13526879,  0.65538305, ...,  0.3379342 ,
         -0.4274403 ,  0.04114941],
        [-0.05847672, -0.13744244,  0.5261469 , ...,  0.34803838,
         -0.33752677, -0.04055665],
        [ 0.04865304, -0.05239412,  0.5647512 , ...,  0.66545975,
         -0.6454591 , -0.13866827]],

       [[ 0.0116337 ,  0.02451276, -0.10758655, ...,  0.15397519,
         -0.08475358, -0.03019335],
        [-0.06115007,  0.32749522, -0.3740465 , ...,  0.41276175,
          0.9359753 ,  0.08835143],
        [-0.33425915,  0.52792215,  0.42075735, ..., -0.45563135,
          1.0646499 ,  1.0318441 ],
        ...,

In [153]:
for c in f.graph.collections:
    print(c, f.graph.get_collection_ref(c))

variables []
local_variables []
trainable_variables []
('__variable_store',) []
('__varscope',) [<tensorflow.python.ops.variable_scope._VariableScopeStore object at 0x7f49dd0a0468>]


In [130]:
input = tf.constant(["hello there"])

vs = []

with tf.GradientTape() as gt:
    encoded = preprocessor(input)

    for v in encoder.variables:
        print("watching", v.name)
        gt.watch(v)

    output = encoder(encoded)

d = gt.gradient(tf.math.reduce_sum(output['sequence_output']), vs)
print(d)

watching word_embeddings/embeddings:0
watching position_embedding/embeddings:0
watching type_embeddings/embeddings:0
watching embeddings/layer_norm/gamma:0
watching embeddings/layer_norm/beta:0
watching transformer/layer_0/self_attention/query/kernel:0
watching transformer/layer_0/self_attention/query/bias:0
watching transformer/layer_0/self_attention/key/kernel:0
watching transformer/layer_0/self_attention/key/bias:0
watching transformer/layer_0/self_attention/value/kernel:0
watching transformer/layer_0/self_attention/value/bias:0
watching transformer/layer_0/self_attention/attention_output/kernel:0
watching transformer/layer_0/self_attention/attention_output/bias:0
watching transformer/layer_0/self_attention_layer_norm/gamma:0
watching transformer/layer_0/self_attention_layer_norm/beta:0
watching transformer/layer_0/intermediate/kernel:0
watching transformer/layer_0/intermediate/bias:0
watching transformer/layer_0/output/kernel:0
watching transformer/layer_0/output/bias:0
watching tr

[]


In [121]:
print(d)

None


In [65]:
@tf.function
def run_model(text):
    encoder_inputs = preprocessor(text)
    outputs = encoder(encoder_inputs)
    return outputs["pooled_output"]

In [86]:
g = run_model.get_concrete_function(input).graph

In [110]:
# g.get_tensor_by_name("word_embeddings/embeddings:0")
g.get_tensor_by_name("transformer/layer_0:0")

KeyError: "The name 'transformer/layer_0:0' refers to a Tensor which does not exist. The operation, 'transformer/layer_0', does not exist in the graph."

In [85]:
encoder_inputs['input_mask'].g

AttributeError: 'KerasTensor' object has no attribute 'g'

In [91]:
for c in g.collections:
    print(c, g.get_collection_ref(c))

variables []
local_variables []
trainable_variables []
('__variable_store',) []
('__varscope',) [<tensorflow.python.ops.variable_scope._VariableScopeStore object at 0x7f52331fdd68>]
local_variablses []
__varscope []


In [101]:
for c in g.outer_graph.collections:
    print(c, g.outer_graph.get_collection_ref(c))

variables []
local_variables []
trainable_variables []
('__variable_store',) []
('__varscope',) [<tensorflow.python.ops.variable_scope._VariableScopeStore object at 0x7f52331fdd68>]


In [106]:
g.outer_graph.as_graph_def()

versions {
  producer: 987
}

In [98]:
g.get_operation_by_name("keras_layer_1/55213").outputs

[<tf.Tensor 'keras_layer_1/55213:0' shape=() dtype=resource>]

In [92]:
for op in g.get_operations():
    print(op)

name: "text"
op: "Placeholder"
attr {
  key: "_user_specified_name"
  value {
    s: "text"
  }
}
attr {
  key: "dtype"
  value {
    type: DT_STRING
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: 1
      }
    }
  }
}

name: "keras_layer/55146"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_RESOURCE
  }
}
attr {
  key: "shape"
  value {
    shape {
    }
  }
}

name: "keras_layer/55148"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT64
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT64
      tensor_shape {
      }
      int64_val: -1
    }
  }
}

name: "keras_layer/StatefulPartitionedCall"
op: "StatefulPartitionedCall"
input: "text"
input: "keras_layer/55146"
input: "keras_layer/55148"
attr {
  key: "Tin"
  value {
    list {
      type: DT_STRING
      type: DT_RESOURCE
      type: DT_INT64
    }
  }
}
attr {
  key: "Tout"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32

In [74]:
vs = g.get_collection_ref(('__varscope',))[0]

In [81]:
vs.current_scope.trainable_variable

[]

In [16]:
model = tf.keras.Model(text_input, sequence_output)
model_encoder = tf.keras.Model(text_input, encoder_inputs)

In [22]:
input = tf.constant(["hello there"])
model_encoder(input)

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

In [23]:
g = encoder.build(input_shape=input.shape)

In [25]:
pooled_output

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer_1')>

In [15]:
tf.gradients(tf.math.reduce_sum(pooled_output[:,0]), encoder_inputs)

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [12]:
# embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
embedding = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

2022-05-05 17:22:02.740923: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.


ValueError: Exception encountered when calling layer "keras_layer_1" (type KerasLayer).

Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * Tensor("inputs:0", shape=(2,), dtype=string)
    * False
    * None
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * {'input_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'), 'input_type_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_type_ids'), 'input_word_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids')}
    * False
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * {'input_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/input_mask'), 'input_word_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='inputs/input_word_ids'), 'input_type_ids': TensorSpec(sha

In [11]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


# Twitter Sentiment Model

[Huggingface](https://huggingface.co/models) offers a variety of pre-trained NLP models to explore. We exemplify in this notebook a [transformer-based twitter sentiment classification model](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment). Before getting started, familiarize yourself with the general Truera API as demonstrated in the [intro notebook using pytorch](intro_demo_pytorch.ipynb).

In [17]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

# Wrap all of the necessary components.
class TwitterSentiment:
    MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

    device = 'cpu'
    # Can also use cuda if available:
    # device = 'cuda:0'

    model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    labels = ['negative', 'neutral', 'positive']

    NEGATIVE = labels.index('negative')
    NEUTRAL = labels.index('neutral')
    POSITIVE = labels.index('positive')

task = TwitterSentiment()

This model quantifies tweets (or really any text you give it) according to its sentiment: positive, negative, or neutral. Lets try it out on some examples.

In [18]:
sentences = ["I'm so happy!", "I'm so sad!", "I cannot tell whether I should be happy or sad!", "meh"]

# Input sentences need to be tokenized first.

inputs = task.tokenizer(sentences, padding=True, return_tensors="pt").to(task.device) # pt refers to pytorch tensor

# The tokenizer gives us vocabulary indexes for each input token (in this case,
# words and some word parts like the "'m" part of "I'm" are tokens).

print(inputs)

# Decode helps inspecting the tokenization produced:

print(task.tokenizer.batch_decode(torch.flatten(inputs['input_ids'])))
# Normally decode would give us a single string for each sentence but we would
# not be able to see some of the non-word tokens there. Flattening first gives
# us a string for each input_id.

{'input_ids': tensor([[   0,  100,  437,   98, 1372,  328,    2,    1,    1,    1,    1,    1,
            1],
        [   0,  100,  437,   98, 5074,  328,    2,    1,    1,    1,    1,    1,
            1],
        [   0,  100, 1395, 1137,  549,   38,  197,   28, 1372,   50, 5074,  328,
            2],
        [   0, 1794,  298,    2,    1,    1,    1,    1,    1,    1,    1,    1,
            1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
['<s>', 'I', "'m", ' so', ' happy', '!', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<s>', 'I', "'m", ' so', ' sad', '!', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<s>', 'I', ' cannot', ' tell', ' whether', ' I', ' should', ' be', ' happy', ' or', ' sad', '!', '</s>', '<s>', 'me', 'h', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

Evaluating huggingface models is straight-forward if we use the structure produced by the tokenizer.

In [19]:
outputs = task.model(**inputs)

print(outputs)

# From logits we can extract the most likely class for each sentence and its readable label.

predictions = [task.labels[i] for i in outputs.logits.argmax(axis=1)]

for sentence, logits, prediction in zip(sentences, outputs.logits, predictions):
    print(logits.to('cpu').detach().numpy(), prediction, sentence)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.3217, -0.8764,  4.0706],
        [ 2.5737, -0.4016, -2.1465],
        [ 0.5973,  0.3778, -0.7691],
        [-0.2267,  0.6011, -0.2009]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[-2.3217378 -0.8763628  4.0705967] positive I'm so happy!
[ 2.5736785  -0.40158105 -2.1465425 ] negative I'm so sad!
[ 0.59733623  0.37779403 -0.7691068 ] negative I cannot tell whether I should be happy or sad!
[-0.22674142  0.601109   -0.20089385] neutral meh


# Model Wrapper

As in the prior notebooks, we need to wrap the pytorch model with the appropriate Trulens functionality. Here we specify the maximum input size (in terms of tokens) each tweet may have.

In [20]:
from trulens.nn.models import get_model_wrapper
from trulens.nn.quantities import ClassQoI
from trulens.nn.attribution import IntegratedGradients
from trulens.nn.attribution import Cut, OutputCut
from trulens.utils.typing import ModelInputs

task.wrapper = get_model_wrapper(task.model, input_shape=(None, task.tokenizer.model_max_length), device=task.device)

INFO: Detected pytorch backend for <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'>.
INFO: Using backend Backend.PYTORCH.
INFO: If this seems incorrect, you can force the correct backend by passing the `backend` parameter directly into your get_model_wrapper call.
DEBUG: Input dtype was not passed in. Defaulting to `torch.float32`.


# Attributions

Applying integrated gradents to the sentiment model is similar as in the prior notebooks except special considerations need to be made for the cuts used as the targets of the attribution (i.e. what do we want to assign importance to). As you may have noted above, the model takes as input integer indexes associated with tokens. As we cannot take gradient with respect to these, we use an alternative: the embedding representation of those same inputs. To instantiate trulens with this regard, we need to find inspect the layer names inside our model:

In [21]:
task.wrapper.print_layer_names()

'roberta_embeddings_word_embeddings':	Embedding(50265, 768, padding_idx=1)
'roberta_embeddings_position_embeddings':	Embedding(514, 768, padding_idx=1)
'roberta_embeddings_token_type_embeddings':	Embedding(1, 768)
'roberta_embeddings_LayerNorm':	LayerNorm((768,), eps=1e-05, elementwise_affine=True)
'roberta_embeddings_dropout':	Dropout(p=0.1, inplace=False)
'roberta_encoder_layer_0_attention_self_query':	Linear(in_features=768, out_features=768, bias=True)
'roberta_encoder_layer_0_attention_self_key':	Linear(in_features=768, out_features=768, bias=True)
'roberta_encoder_layer_0_attention_self_value':	Linear(in_features=768, out_features=768, bias=True)
'roberta_encoder_layer_0_attention_self_dropout':	Dropout(p=0.1, inplace=False)
'roberta_encoder_layer_0_attention_output_dense':	Linear(in_features=768, out_features=768, bias=True)
'roberta_encoder_layer_0_attention_output_LayerNorm':	LayerNorm((768,), eps=1e-05, elementwise_affine=True)
'roberta_encoder_layer_0_attention_output_dropou

## Parameters

Above, `roberta_embeddings_word_embeddings` is the layer that produces a continuous representation of each input token so we will use that layer as the one defining the **distribution of interest**. While most neural NLP models contain a token embedding, the layer name will differ.

The second thing to note is the form of model outputs. Specifically, outputs are structures which contain a 'logits' attribute that stores the model scores.

Putting these things together, we instantiate `IntegratedGradients` to attribute each embedding dimension to the maximum class (i.e. the predicted class).

In [22]:
infl_max = IntegratedGradients(
    model = task.wrapper,
    doi_cut=Cut('roberta_embeddings_word_embeddings'),
    qoi_cut=OutputCut(accessor=lambda o: o['logits'])
)

In [23]:
# Alternatively we can look at a particular class:

infl_positive = IntegratedGradients(
    model = task.wrapper,
    doi_cut=Cut('roberta_embeddings_word_embeddings'),
    qoi=ClassQoI(task.POSITIVE),
    qoi_cut=OutputCut(accessor=lambda o: o['logits'])
)

Getting attributions uses the same call as model evaluation.

In [24]:
attrs = infl_max.attributions(**inputs)

for token_ids, token_attr in zip(inputs['input_ids'], attrs):
    for token_id, token_attr in zip(token_ids, token_attr):
        # Not that each `word_attr` has a magnitude for each of the embedding
        # dimensions, of which there are many. We aggregate them for easier
        # interpretation and display.
        attr = token_attr.sum()

        word = task.tokenizer.decode(token_id)

        print(f"{word}({attr:0.3f})", end=' ')

    print()

<s>(0.057) I(0.202) 'm(0.220)  so(0.279)  happy(0.506) !(0.106) </s>(-0.078) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) 
<s>(0.022) I(0.039) 'm(-0.143)  so(0.168)  sad(-0.019) !(-0.149) </s>(-0.044) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) 
<s>(-0.028) I(0.002)  cannot(-0.237)  tell(-0.018)  whether(0.125)  I(-0.065)  should(0.021)  be(-0.048)  happy(-0.133)  or(-0.156)  sad(-0.452) !(-0.161) </s>(0.001) 
<s>(0.041) me(-0.259) h(-0.384) </s>(-0.089) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) <pad>(0.000) 


A listing as above is not very readable so Trulens comes with some utilities to present token influences a bit more concisely. First we need to set up a few parameters to make use of it:

In [25]:
from trulens.visualizations import NLP

V = NLP(
    wrapper=task.wrapper,
    labels=task.labels,
    decode=lambda x: task.tokenizer.decode(x),
    tokenize=lambda sentences: ModelInputs(kwargs=task.tokenizer(sentences, padding=True, return_tensors='pt')).map(lambda t: t.to(task.device)),
    # huggingface models can take as input the keyword args as per produced by their tokenizers.

    input_accessor=lambda x: x.kwargs['input_ids'],
    # for huggingface models, input/token ids are under input_ids key in the input dictionary

    output_accessor=lambda x: x['logits'],
    # and logits under 'logits' key in the output dictionary

    hidden_tokens=set([task.tokenizer.pad_token_id])
    # do not display these tokens
)

print("QOI = MAX PREDICTION")
display(V.token_attribution(sentences, infl_max))

print("QOI = POSITIVE")
display(V.token_attribution(sentences, infl_positive))

QOI = MAX PREDICTION


QOI = POSITIVE


# Baselines

We see in the above results that special tokens such as the sentence end **&lt;/s&gt;** contributes are found to contribute a lot to the model outputs. While this may be useful in some contexts, we are more interested in the contributions of the actual words in these sentences. To focus on the words more, we need to adjust the **baseline** used in the integrated gradients computation. By default in the instantiation so far, the baseline for each token is a zero vector of the same shape as its embedding. By making the basaeline be identicaly to the explained instances on special tokens, we can rid their impact from our measurement. Trulens provides a utility for this purpose in terms of `token_baseline` which constructs for you the methods to compute the appropriate baseline. 

In [26]:
from trulens.utils.nlp import token_baseline

inputs_baseline_ids, inputs_baseline_embeddings = token_baseline(
    keep_tokens=set([task.tokenizer.cls_token_id, task.tokenizer.sep_token_id]),
    # Which tokens to preserve.

    replacement_token=task.tokenizer.pad_token_id,
    # What to replace tokens with.

    input_accessor=lambda x: x.kwargs['input_ids'],

    ids_to_embeddings=task.model.get_input_embeddings()
    # Callable to produce embeddings from token ids.
)

We can now inspect the baselines on some example sentences. The first method returned by `token_baseline` gives us token ids to inspect while the second gives us the embeddings of the baseline which we will pass to the attributions method.

In [27]:
print("originals=", task.tokenizer.batch_decode(inputs['input_ids']))

baseline_word_ids = inputs_baseline_ids(model_inputs=ModelInputs(args=[], kwargs=inputs))
print("baselines=", task.tokenizer.batch_decode(baseline_word_ids))

originals= ["<s>I'm so happy!</s><pad><pad><pad><pad><pad><pad>", "<s>I'm so sad!</s><pad><pad><pad><pad><pad><pad>", '<s>I cannot tell whether I should be happy or sad!</s>', '<s>meh</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>']
baselines= ['<s><pad><pad><pad><pad><pad></s><pad><pad><pad><pad><pad><pad>', '<s><pad><pad><pad><pad><pad></s><pad><pad><pad><pad><pad><pad>', '<s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad></s>', '<s><pad><pad></s><pad><pad><pad><pad><pad><pad><pad><pad><pad>']


In [28]:
infl_positive_baseline = IntegratedGradients(
    model = task.wrapper,
    resolution=50,
    baseline = inputs_baseline_embeddings,
    doi_cut=Cut('roberta_embeddings_word_embeddings'),
    qoi=ClassQoI(task.POSITIVE),
    qoi_cut=OutputCut(accessor=lambda o: o['logits'])
)

print("QOI = POSITIVE WITH BASELINE")
display(V.token_attribution(sentences, infl_positive_baseline))

QOI = POSITIVE WITH BASELINE


As we see, the baseline eliminated the measurement of contribution of the special tokens.